In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import yaml
from pathlib import Path
from IPython.core.display import HTML
from IPython.display import display

from oml.lightning.entrypoints.validate import pl_val
from oml.lightning.callbacks.metric import MetricValCallback
from oml.const import MOCK_DATASET_PATH
from oml.utils.download_mock_dataset import download_mock_dataset, check_mock_dataset_exists

display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 330)

%matplotlib inline

In [ ]:
if not check_mock_dataset_exists(MOCK_DATASET_PATH):
    download_mock_dataset(MOCK_DATASET_PATH);

In [ ]:
cfg = f"""
    dataset_root: {MOCK_DATASET_PATH}
    dataframe_name: df_with_bboxes.csv
    logs_root: logs
    bs_val: 128
    num_workers: 2
    
    transforms_val:
      name: norm_resize_albu
      args:
        im_size: 224

    model:
      name: vit
      args:
        arch: vits16
        normalise_features: False
        use_multi_scale: False
        weights: vits16_dino

    metric_args:
      cmc_top_k: [1]
      map_top_k: [5]
      
    accelerator: cpu
    devices: 1
  
"""
        
trainer, ret_dict = pl_val(yaml.load(cfg, Loader=yaml.Loader));

clb_metric = [x for x in trainer.callbacks if isinstance(x, MetricValCallback)][0]

In [ ]:
n_query = min(10, clb_metric.metric.distance_matrix.shape[0])
fig = clb_metric.metric.get_plot_for_worst_queries(metric_name='OVERALL/map/5', topk_queries=n_query, topk_instances=4, verbose=True)